In [2]:
#[allow(unused_variables)]

# Lifetimes & Annotations

## Types of &str

* **String literals** - they last for the whole program (they are written directly into the binary)
  * type - `&'static str`

In [3]:
let name: &'static str = "John Smith";

* **Borrowed `str`** - borrowed reference to `String` object (without `'static` lifetime)
  * conversion from `String` to `&str` is allowed thanks to a `Deref` trait
  * we can always pass reference to `String` to a function that expects `&str` - it will be automatically converted (*type coercion* - *deref coercion*)

In [4]:
fn print_description(description: &str) {
    println!("{}", description);
}

fn main() {
    let description: String = "And now something completly different...".to_string();
    let _borrowed_description: &str = &description; 

    print_description(&description); // deref coercion works here
}

main();

And now something completly different...


## Returning references

* You can’t return a reference to local variable that only lives inside a function - it would be a *dangling reference*

In [5]:
fn create_dangling_reference() -> &'static str {
    let s = String::from("Hello, world!");
    &s
}

Error: cannot return reference to local variable `s`

* You can return a reference to static variable (like string literals) - return type needs `'static` *lifetime annotation*

In [6]:
fn get_greetings() -> &'static str {
    let s = "Hello, world!";
    s
}

## Lifetime annotations

* Most of the time, Rust takes care of lifetimes for you, but sometimes, it needs a bit of extra help. This extra help is called a **lifetime annotation**, which means “extra lifetime information.”
* Liftetime annotations don't change the lifetimes of any references, they just describe the relationships of the lifetimes of multiple references.
* Lifetime annotations syntax: the names of lifetime parameters must start with an apostrophe (') and are usually all lowercase and very short, like this: `'a`, `'b`, and so on. But the names of lifetime parameters are arbitrary and have no meaning.

### Inferring lifetimes

* Rust has a **lifetime elision** rules, which are a set of three rules that the compiler follows to figure out what lifetimes references have when there aren't explicit annotations.
* The first rule applies to input lifetimes, and the second and third rules apply to output lifetimes. If the compiler gets to the end of the three rules and there are still references for which it can’t figure out lifetimes, the compiler will stop with an error.

* **The first rule**: each parameter that is a reference gets its own lifetime parameter. In other words, a function with one parameter gets one lifetime parameter: `fn foo<'a>(x: &'a i32);` a function with two parameters gets two separate lifetime parameters: `fn foo<'a, 'b>(x: &'a i32, y: &'b i32);` and so on.
* **The second rule**: if there is exactly one input lifetime parameter, that lifetime is assigned to all output lifetime parameters: `fn foo<'a>(x: &'a i32) -> &'a i32;`
* **The third rule**: if there are multiple input lifetime parameters, but one of them is `&self` or `&mut self` because this is a method, the lifetime of `self` is assigned to all output lifetime parameters: `fn foo<'a, 'b>(&'a self, x: &'b i32) -> &'a i32;`

In [7]:
fn foo(text: &str) {
    println!("{}", text);
}

can be explicitly written as:

In [8]:
fn foo<'a>(text: &'a str) {
    println!("{}", text);
}

let text = String::from("Hello, world!");
foo(&text);

Hello, world!


From the `foo` signature Rust knows it will not save `text` anywhere that might outlive the function call: any lifetime that encloses the call must work for `'a`.

In [9]:
static mut BAG: Vec<&str> = Vec::<&str>::new();

fn bar<'a>(text: &'a str) {
    unsafe {
        BAG.push(text);
    }
}

fn main() {
    let text = String::from("Hello, world!");
    bar(&text);
}

Error: borrowed data escapes outside of function

Acceptable lifetime for text is `static`:

In [10]:
static mut BAG: Vec<&str> = Vec::<&str>::new();

fn bar(text: &'static str) {
    unsafe {
        BAG.push(text);
    }
}

fn main() {
    let text = "Hello, world!";
    bar(&text);
}

### Returning references

When a function takes a single reference as an argument, and returns a single reference, Rust assumes that the two must have the same lifetime (see *the second rule*).

```rust

In [11]:
fn ref_to_min(data: &mut [i32]) -> &mut i32 {
    // let min = data.iter_mut().min().unwrap();
    
    let mut index: usize = 0;
    for (i, value) in data.iter().enumerate() {
        if *value < data[index] {
            index = i;
        }
    } 

    &mut data[index]
}

In [12]:
fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    *the_smallest = 0;

    println!("{:?}", data);
}

main();

[42, 665, 0, 44, 99]


In [13]:
let s;

{
    let mut data = [9, 4, 1, 0, 1, 4, 9];
    s = ref_to_min(&mut data);
}

println!("{}", *s);

Error: `data` does not live long enough

## Structs Containing References (Annotations)

In [14]:
struct View {
    value: &i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);

    let view = View { value: the_smallest };
    println!("{}", view.value);
}

Error: missing lifetime specifier

Whenever a reference type appears inside another type’s definition, you must write out its lifetime.

In [15]:
struct View<'a> {
    value: &'a i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    let view = View { value: the_smallest };
    println!("{}", view.value);
}

main();

1


Now the `View` type has a lifetime, just as reference types do. Each value you create of type `Value` gets a fresh lifetime `'a`, which becomes constrained by how you use the value. The lifetime of any reference you store in `ref_value` had better enclose `'a`, and `'a` must outlast the lifetime of wherever you store the `View`.

In [16]:
struct View<'a> {
    value: &'a i32
}

#[allow(unused_variables)]
fn main() {
    let mut bag: Vec<View> = Vec::new();
    {
        let mut data = [42, 665, 1, 44, 99];
        let the_smallest: &mut i32 = ref_to_min(&mut data);

        let view = View { ref_value: the_smallest };
        println!("{}", view.ref_value);
        bag.push(view);
    }

    println!("{}", bag[0].ref_value);
}

main();

Error: struct `View<'_>` has no field named `ref_value`

Error: no field `ref_value` on type `View<'_>`

Error: no field `ref_value` on type `View<'_>`

When we want to place `View` inside of another `struct`, we must specify the lifetime of the reference it contains:

In [17]:
struct Record {
    v: View<'static>
}

// or

struct MinView<'a> {
    v: View<'a>
}

A type’s lifetime parameters always reveal whether it contains references with interesting (that is, non-`'static`) lifetimes, and what those lifetimes can be

In [18]:
fn get_min<'a>(data: &'a mut [i32]) -> MinView<'a> {
    let min: &'a mut i32 = data.iter_mut().min().unwrap();
    MinView { v: View { value: min } }
}

Without looking into the definition of the `MinView` type at all, we can tell that, if we receive a `MinView` from `get_min`, whatever references it contains must point into the input slice we passed in, and nowhere else (except perhaps at 'static values).

## Distinct Lifetimes


In [19]:
struct Record<'a> {
    x: &'a i32,
    y: &'a i32
}

In [20]:
#[allow(unused_variables)]
fn main() {
    let x = 42;
    let r;
    {
        let y = 665;
        let local = Record { x: &x, y: &y };
        r = local.x;    
    }
    println!("{}", r);
}

main();

Error: `y` does not live long enough

This code doesn’t create any dangling pointers. The reference to `y` stays in `local`, which goes out of scope before `y` does. The reference to `x` ends up in `r`, which doesn’t outlive `x`.

If you try to compile this, however, Rust will complain that `y` does not live long enough, even though it clearly does. Why is Rust worried? If you work through the code carefully, you can follow its reasoning:

* Both fields of `Record` are references with the same lifetime `'a`, so Rust must find a single lifetime that works for both `local.x` and `local.y`.
* We assign `r = local.x`, requiring `'a` to enclose `r’s` lifetime.
* We initialized `local.y` with `&y`, requiring `'a` to be no longer than `y`’s lifetime.

These constraints are impossible to satisfy: no lifetime is shorter than `y`’s scope, but longer than `r`’s.

The problem arises because both references in `Record` have the same lifetime `'a`. Changing the definition of `Record` to let each reference have a distinct lifetime fixes everything:

In [21]:
struct Record<'a, 'b> {
    x: &'a i32,
    y: &'b i32,
}

impl<'a, 'b> Record<'a, 'b> {
    fn print(&self) {
        println!("x: {}, y: {}", self.x, self.y);
    }
}

In [22]:
#[allow(unused_variables)]
fn main() {
    let x = 42;
    let r;
    {
        let y = 665;
        let local = Record { x: &x, y: &y };
        r = local.x;
        local.print();
    }
    println!("{}", r);
}

main();

x: 42, y: 665
42


With this definition, `local.x` and `local.y` have independent lifetimes. What we do with `local.x` has no effect on what we store in `local.y`, so it’s easy to satisfy the constraints now: `'a` can simply be `r`’s lifetime, and `'b` can be `local`’s. (`y`’s lifetime would work too for `'b`, but Rust tries to choose the smallest lifetime that works). 

Function signatures can have similar effects. Suppose we have a function like this:

In [23]:
fn foobar_tight<'a>(x: &'a i32, y: &'a i32) -> &'a i32 { // perhaps too restrictive
    x
}

In [24]:
fn foobar<'a, 'b>(x: &'a i32, y: &'b i32) -> &'a i32 { // looser
    x
}

Try the simplest possible definition first, and then loosen restrictions until the code compiles. Since Rust won’t permit the code to run unless it’s safe, simply waiting to be told when there’s a problem is a perfectly acceptable tactic.

# Sharing vs. Mutation

In [25]:
fn extend(vec: &mut Vec<i32>, slice: &[i32]) {
    for item in slice {
        vec.push(*item);
    }
}

let mut data = vec![1, 2, 3, 4];
let exta_data_1 = vec![5, 6, 7];
let exta_data_2 = vec![8, 9, 10];

extend(&mut data, &exta_data_1);
extend(&mut data, &exta_data_2);

println!("{:?}", data);

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Let's try extend a vector with a slice of its own elements:

In [26]:
extend(data, &data);

Error: mismatched types

Error: unused variable: `y`

Error: unused variable: `y`

## Rust Rules for Mutation & Sharing

* **Shared access** is **read-only access**. Values borrowed by shared references are read-only. Across the lifetime of a shared reference, neither its referent, nor anything reachable from that referent, can be changed by anything. There exist no live mutable references to anything in that structure; its owner is held read-only; and so on. It’s immutable

* **Mutable access** is **exclusive access**. A value borrowed by a mutable reference is reachable exclusively via that reference. Across the lifetime of a mutable reference, there is no other usable path to its referent, or to any value reachable from there. The only references whose lifetimes may overlap with a mutable reference are those you borrow from the mutable reference itself.

In [2]:
{
    let mut x = 10;
    let rx1 = &x;
    let rx2 = &x;     // OK: multiple shared borrows permitted

    x += 10;         // Error: cannot assign to `x` because it is borrowed

    println!("{x}, {rx1}, {rx2}");
}

Error: cannot assign to `x` because it is borrowed

In [3]:
{
    let mut x = 10;
    let rx = &x;
    
    let mx = &mut x; // Error: cannot borrow `x` as mutable because it is also borrowed as immutable

    println!("{x}, {rx}, {mx}");
}

Error: cannot borrow `x` as mutable because it is also borrowed as immutable

Error: cannot borrow `x` as immutable because it is also borrowed as mutable

In [ ]:
{
    let mut y = 20;
    let my1 = &mut y;
    let my2 = &mut y;  // Error: cannot borrow as mutable more than once

    println!("{y}, {my1}, {my2}");
}

Error: cannot borrow `y` as mutable more than once at a time

Error: cannot borrow `y` as immutable because it is also borrowed as mutable

It is OK to reborrow a shared reference from a shared reference:

In [ ]:
{
    let record = (42, 665);
    let r = &record;
    let r0 = &r.0;  // OK: reborrowing shared as shared
}

()

In [7]:
{
    let record = (42, 665);
    let r = &record;
    let m1 = &mut r.1;  // Error: cannot reborrow as mutable because it is also borrowed as immutable
}

Error: unused variable: `m1`

Error: cannot borrow `r.1` as mutable, as it is behind a `&` reference

You can reborrow from a mutable reference:

In [12]:
{
    let mut record = (42, 665);
    let m = &mut record;
    let m0 = &mut m.0; 
    *m0 = 123; // OK: mutable borrow is unique

    let r1 = &m.1; // Ok: reborrowing mutable as shared
    println!("{m0}, {r1}");
}

123, 665


()